<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastián Ojeda G.


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `https://github.com/sebajedi23/MDS7202`

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.2 MB/s eta 0:00:00


In [2]:
#Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import make_scorer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor

#Optuna
import optuna
from optuna.integration import XGBoostPruningCallback
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_param_importances

#Otros
import pandas as pd
import numpy as np
from datetime import datetime
from xgboost import XGBRegressor
import pickle
import warnings

In [3]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Colab Notebooks/Laboratorio de programación científica para ciencia de datos/Lab 9/'
except:
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [4]:
#Creamos un DataFrame a partir de los datos
df = pd.read_csv(path + 'sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [5]:
#1. Separamos los datos en conjuntos de train (70%), validation (20%) y test (10%)
#Dividimos en train y test con proporción 70/30
train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)

#Dividimos test en test y validation, con test 1/3 (10% del total)
test_data, val_data = train_test_split(test_data, test_size=1/3, random_state=42)

In [6]:
#2. Implementamos un FunctionTransformer para extraer el día, mes y año de la variable date
def extract_date_info(df):
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    return df.drop('date', axis=1)

#Creamos el FunctionTransformer
date_transformer = FunctionTransformer(extract_date_info, validate=False)

In [7]:
#3. Implementamos un ColumnTransformer para procesar los datos numéricos y categóricos
#Creamos una lista con las columnas numéricas
numeric_features = ['lat', 'long', 'pop', 'price']

#Creamos una lista con las columnas categóricas
categorical_features = ['city', 'shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']

#Definimos el pre-procesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [8]:
#4. Guardamos los pasos anteriores en un Pipeline con DummyRegressor
pipeline_dummy = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor())
])

In [9]:
#5. Entrenamos el pipeline y reportar la métrica mean_absolute_error sobre los datos de validación
#Dividimos los datos en X e Y de acuerdo a la columna "quantity"
X_train = train_data.drop('quantity', axis=1)
y_train = train_data['quantity']
X_val = val_data.drop('quantity', axis=1)
y_val = val_data['quantity']
X_test = test_data.drop('quantity', axis=1)
y_test = test_data['quantity']

#Entrenamos el pipeline con los datos de train
pipeline_dummy.fit(X_train, y_train)

#Predecimos sobre los datos de validation
val_predictions_dummy = pipeline_dummy.predict(X_val)

#Obtenemos el valor del MAE
mae_dummy = mean_absolute_error(y_val, val_predictions_dummy)
print(f'MAE con Dummy Regressor: {mae_dummy}')

MAE con Dummy Regressor: 13413.17673026018


¿Cómo se interpreta esta métrica para el contexto del negocio?

**Respuesta:**
````
En el contexto del negocio, el MAE o (Mean Squared Error) representa el error promedio absoluto entre las predicciones y los
valores reales, es decir, la cantidad promedio de unidades vendidas por la cual las predicciones difieren de las ventas reales.
Cuanto menor sea el MAE, mejor será el rendimiento del modelo, esto ya que las predicciones diferirán menos de los valores reales.

````


In [10]:
#6. Entrenamos el Pipeline con XGBRegressor y revisaremos si existen cambios en el MAE
#Creamos el pipeline con transformador, pre-procesador y regresor
pipeline_xgb = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

#Entrenamos el pipeline con los datos de train
pipeline_xgb.fit(X_train, y_train)

#Predecimos sobre los datos de validation
val_predictions_xgb = pipeline_xgb.predict(X_val)

#Obtenemos el valor del MAE
mae_xgb = mean_absolute_error(y_val, val_predictions_xgb)
print(f'MAE con XGBoost: {mae_xgb}')

MAE con XGBoost: 2427.1128332224994


¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el "DummyRegressor"?

**Respuesta:**
````
Se observa que el MAE con XGBoost es significativamente menor que el MAE con Dummy Regressor, llegando a un valor más de un 80%
menor. Esto quiere decir que el modelo XGBoost está haciendo predicciones más precisas en comparación con el promedio de las
respuestas de entrenamiento.

Lo anterior quiere decir que el desempeño de XGBoost es mucho mejor que el del DummyRegressor, teniendo en promedio diferencias de
demanda inferiores a 2500 unidades, respecto a las más de 13000 del DummyRegressor. Esto quiere decir que los valores predichos se
acercan más a los reales, teniendo por ende una respuesta más acorde a lo esperado

````

In [11]:
#7. Guardamos ambos modelos en un archivo .pkl
#Guardamos el regresor dummy
with open('regressor_dummy.pkl', 'wb') as f:
    pickle.dump(pipeline_dummy, f)

#Guardamos el regresor XGBoost
with open('regressor_xgb.pkl', 'wb') as f:
    pickle.dump(pipeline_xgb, f)

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [12]:
#Re-imprimimos los MAE sin implementar la relación monótona negativa
print(f'MAE con Dummy Regressor: {mae_dummy}')
print(f'MAE con XGBoost: {mae_xgb}')

#Definimos la relación monótona negativa entre demanda y precio
monotone_constraints = (0, 0, 0, -1)

#Creamos el pipeline con transformador, pre-procesador y regresor
pipeline_xgb_constraints = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(monotone_constraints=monotone_constraints))
])

#Entrenamos el pipeline con los datos de train
pipeline_xgb_constraints.fit(X_train, y_train)

#Predecimos sobre los datos de validation
val_predictions_xgb_with_constraints = pipeline_xgb_constraints.predict(X_val)

#Obtenemos el valor del MAE
mae_xgb_with_constraints = mean_absolute_error(y_val, val_predictions_xgb_with_constraints)
print(f'MAE con XGBoost y relación monótona negativa: {mae_xgb_with_constraints}')

#Guardamos el regresor XGBoost con relación monótona negativa
with open('regressor_xgb_constraints.pkl', 'wb') as f:
    pickle.dump(pipeline_xgb_constraints, f)

MAE con Dummy Regressor: 13413.17673026018
MAE con XGBoost: 2427.1128332224994
MAE con XGBoost y relación monótona negativa: 2447.6054746407926


¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

**Respuesta:**
```
Con los resultados obtenidos podemos observar que el MAE aumenta de 2427.11 a 2447.61 al aplicar la relación monótona negativa entre
precio y demanda. Lo anterior indica que el modelo XGBoost ajustado con la restricción monótona negativa tiene un rendimiento
levemente peor en términos de predicción de la demanda en comparación al modelo XGBoost sin restricciones.

Lo anterior quiere decir que la relación sugerida entre demanda y precio no es real o no es relevante para el problema, puesto que no
afecta mayormente los resultados obtenidos y el poco cambio que genera es negativo, por lo que es mejor desestimar la recomendación
de nuestro colega aficionado a la economía ya que no tiene la razón.
```

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

---
Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?

**Respuesta:**
```
Los hiperparámetros y sus roles son los siguientes:
learning_rate: Define la tasa de aprendizaje del modelo.
n_estimators: Número de árboles (estimadores) en el ensamble.
max_depth: Profundidad máxima de cada árbol.
max_leaves: Número máximo de hojas por árbol.
min_child_weight: Mínimo peso requerido para crear un nuevo nodo en el árbol.
reg_alpha: Término de regularización L1.
reg_lambda: Término de regularización L2.

Respecto a los rangos de optimización, estos tienen sentido ya que ofrecen una buena flexibilidad para ajustar los hiperparámetros
según las características específicas del problema, por ejemplo el rango de learning_rate favorece un aprendizaje gradual, evitando
un overfitting, al igual que con el caso de max_depth, reg_alpha y reg_lambda. Por otro lado, n_estimators, max leaves y
min_child_weight ofrecen un rango ideal para que el modelo puede ajustarse a diferentes posibles niveles de complejidad del problema.
```

In [25]:
#Fijamos la semilla para asegurar consistencia entre ejecuciones
np.random.seed(42)
#Definimos la función objetivo
def objective(trial):
    #Definimos los rangos de los hiperparámetros a optimizar
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
    }

    #Definimos los rangos de min_frequency OneHotEncoder
    onehot_params = {
        'min_frequency': trial.suggest_float('min_frequency', 0.0, 1.0)
    }

    #Creamos el pipeline con los hiperparámetros obtenidos
    pipeline_xgb = Pipeline([
        ('date_transformer', date_transformer),
        ('preprocessor', ColumnTransformer(
            transformers=[
                ('num', MinMaxScaler(), numeric_features),
                ('cat', OneHotEncoder(handle_unknown='ignore',
                                      sparse_output=False,
                                      **onehot_params),
                 categorical_features)
            ],
            remainder="passthrough",
            verbose_feature_names_out=False
        )),
        ('regressor', XGBRegressor(seed=42,
                                   **xgb_params))
    ])

    #Entrenamos el pipeline con los datos de train
    pipeline_xgb.fit(X_train, y_train)

    #Predecimos sobre los datos de validation
    val_predictions_xgb = pipeline_xgb.predict(X_val)

    #Calculamos el MAE
    mae_xgb = mean_absolute_error(y_val, val_predictions_xgb)

    return mae_xgb

In [26]:
#Configuramos el optimizador de Optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
sampler = TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)

#Ejecutamos el estudio de optimización
study.optimize(objective, timeout=300, show_progress_bar=True)

#Obtenemos los mejores hiperparámetros y el MAE
best_params = study.best_params
best_mae = study.best_value
num_trials = len(study.trials)

#Reportamos los resultados relevantes
print(f'Número de trials: {num_trials}')
print(f'MAE con los mejores hiperparámetros: {best_mae}')
print(f'Mejores hiperparámetros: {best_params}')

   0%|          | 00:00/05:00

Número de trials: 135
MAE con los mejores hiperparámetros: 2043.054916259111
Mejores hiperparámetros: {'learning_rate': 0.06685908862102091, 'n_estimators': 803, 'max_depth': 9, 'max_leaves': 48, 'min_child_weight': 3, 'reg_alpha': 0.0012829997393770387, 'reg_lambda': 0.11839783284335671, 'min_frequency': 0.05743884972235069}


In [27]:
#Guardamos los mejores hiperparámetros de XGBoost obtenidos en un diccionario
params_xgb = {'learning_rate': best_params['learning_rate'],
              'n_estimators': best_params['n_estimators'],
              'max_depth': best_params['max_depth'],
              'max_leaves': best_params['max_leaves'],
              'min_child_weight': best_params['min_child_weight'],
              'reg_alpha': best_params['reg_alpha'],
              'reg_lambda': best_params['reg_lambda'],}


#Definimos un pipeline con los mejores hiperparámetros de XGBoost y OneHotEncoder
best_model = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', ColumnTransformer(
        transformers=[
                      ('num', MinMaxScaler(), numeric_features),
                      ('cat', OneHotEncoder(handle_unknown='ignore',
                                            sparse_output=False,
                                            min_frequency=best_params['min_frequency']),
                      categorical_features)
                      ],
        remainder="passthrough",
        verbose_feature_names_out=False
    )),
    ('regressor', XGBRegressor(seed=42,**params_xgb))
])

#Entrenamos el modelo con los datos de train
best_model.fit(X_train, y_train)

#Predecimos sobre los datos de validation
val_predictions_xgb_optuna = best_model.predict(X_val)

#Obtenemos el MAE
mae_xgb_optuna = mean_absolute_error(y_val, val_predictions_xgb_optuna)

#Reportamos los resultados relevantes
print(f'MAE con los mejores hiperparámetros: {mae_xgb_optuna}')
print(f'Número de trials: {num_trials}')
print('\nMejores hiperparámetros:')
for key, value in best_params.items():
    print(f'{key}: {value}')

#Guardamos el regresor XGBoost con relación monótona negativa
with open('regressor_xgb_optuna.pkl', 'wb') as f:
    pickle.dump(best_model, f)

MAE con los mejores hiperparámetros: 2043.054916259111
Número de trials: 135

Mejores hiperparámetros:
learning_rate: 0.06685908862102091
n_estimators: 803
max_depth: 9
max_leaves: 48
min_child_weight: 3
reg_alpha: 0.0012829997393770387
reg_lambda: 0.11839783284335671
min_frequency: 0.05743884972235069


Reportar el número de trials, el MAE y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?

**Respuesta:**
```
En total se realizaron 135 trials, teniendo un mejor MAE de 2043, con los hiperparámetros reportados anteriormente.
Los resultados mejoran respecto a la sección anterior, pasando de un MAE de 2427.11 a uno de 2043.05, esto se debe a que Optuna busca
en el espacio de hiperparámetros entregado con el objetivo de buscar la combinación de estos que minimice el MAE. Es posible que
estos resultados pudieran haber mejorado aun más si no se tuviera el timeout de 5 minutos, sin embargo la mejora es apreciable con
este tiempo por lo que se considera que los resultados obtenidos son preferibles a los anteriores.
```

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

---
¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?

**Respuesta:**
```
El pruning o poda en optimización de hiperparámetros se refiere a cortar (o podar) un entrenamiento cuando este no está convergiendo
hacia un rendimiento aceptable. Esto puede ahorrar tiempo de cómputo al detener la ejecución de entrenamientos que llevarán a
resultados contraproducentes.

Esta estrategia debería mejorar los resultados anteriores, ya que ayuda a evitar el sobreajuste y reducir el tiempo de entrenamiento
ya que si un modelo no mejora significativamente después de un cierto número de iteraciones o trials, se detendrá la ejecución,
evitando un sobreajuste y evitando ejecutar más trials que no llevarán a mejoras significativas.
```

In [16]:
#Definimos la función objetivo con pruning
def objective_prunners(trial):
    #Definimos los rangos de los hiperparámetros a optimizar
    xgb_params = {
        'eval_metric': mean_absolute_error,
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
    }

    #Definimos los rangos de min_frequency OneHotEncoder
    onehot_params = {
        'min_frequency': trial.suggest_float('min_frequency', 0.0, 1.0)
    }

    #Configuramos el pruner de optuna
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial,
                                                                 observation_key="validation_0-mean_absolute_error")

    #Creamos el ColumnTransformer para el pre-procesamiento
    preprocessor_opt = ColumnTransformer(
        transformers=[
                      ('num', MinMaxScaler(), numeric_features),
                      ('cat', OneHotEncoder(handle_unknown='error',
                                            sparse_output=False,
                                            **onehot_params),
                      categorical_features),
                      ])

    #Mantenemos el formato de pandas
    preprocessor_opt.set_output(transform='pandas')

    #Definimos un pipeline para el pre-procesamiento
    pipeline_procesamiento = Pipeline(steps=[
        ('date_extraction', date_transformer),
        ('preprocessor', preprocessor_opt),
    ])

    #Definimos un segundo pipeline con el regresor
    pipeline_xgb = Pipeline([
        ('date_transformer', date_transformer),
        ('preprocessor',preprocessor_opt),
        ('regressor', XGBRegressor(seed=42,**xgb_params))
    ])

    #Entrenamos el pipeline con los datos de train
    pipeline_procesamiento.fit_transform(X_train)

    # Aplicar la misma transformación al conjunto de validación
    X_val_processed = pipeline_procesamiento.transform(X_val)

    #Entrenamos el pipeline con pruning en los datos de train
    pipeline_xgb.fit(X_train,
                     y_train,
                     regressor__callbacks=[pruning_callback],
                     regressor__eval_set=[(X_val_processed,y_val),],
                     regressor__verbose=False)

    #Predecimos sobre los datos de validation
    val_predictions_xgb = pipeline_xgb.predict(X_val)

    #Obtenemos el MAE
    mae_xgb = mean_absolute_error(y_val, val_predictions_xgb)
    return mae_xgb

In [17]:
#Ocultamos las advertencias
warnings.filterwarnings("ignore")

#Configuramos el optimizador de Optuna con pruning
sampler = TPESampler(seed=42)
study_opt_pr = optuna.create_study(direction='minimize', sampler=sampler)

#Ejecutamos el estudio de optimización con pruning
study_opt_pr.optimize(objective_prunners,
                      timeout=300,
                      show_progress_bar = True)

#Obtenemos los mejores hiperparámetros y el MAE
best_params = study_opt_pr.best_params
best_mae = study_opt_pr.best_value
num_trials = len(study_opt_pr.trials)

#Reportamos los resultados relevantes
print(f'Número de trials: {num_trials}')
print(f'MAE con los mejores hiperparámetros: {best_mae}')
print(f'Mejores hiperparámetros: {best_params}')

   0%|          | 00:00/05:00

Número de trials: 140
MAE con los mejores hiperparámetros: 1940.9989838382833
Mejores hiperparámetros: {'learning_rate': 0.09391585041713191, 'n_estimators': 956, 'max_depth': 10, 'max_leaves': 84, 'min_child_weight': 1, 'reg_alpha': 0.8105922236983762, 'reg_lambda': 0.4876393145933732, 'min_frequency': 0.05401631954675916}


In [18]:
# Obtener el mejor modelo con los mejores hiperparámetros
params_xgb_prunning = {
                      'learning_rate': best_params['learning_rate'],
                      'n_estimators': best_params['n_estimators'],
                      'max_depth': best_params['max_depth'],
                      'max_leaves': best_params['max_leaves'],
                      'min_child_weight': best_params['min_child_weight'],
                      'reg_alpha': best_params['reg_alpha'],
                      'reg_lambda': best_params['reg_lambda'],
                      }


preprocessor_opt = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='error',sparse_output=False, min_frequency=best_params['min_frequency']), categorical_features),
        ])

best_model = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor_opt),
    ('regressor', XGBRegressor(seed=42,**params_xgb_prunning))
])

# Entrenar el mejor modelo con todos los datos
best_model.fit(X_train, y_train)
val_predictions_xgb_prunning = best_model.predict(X_val)
mae_xgb_prunning = mean_absolute_error(y_val, val_predictions_xgb_prunning)
print(f'MAE con XGBoost Optimizado y Prunning: {mae_xgb_prunning}')
print(f'Número de trials: {num_trials}')
# Mostramos los hiperparámetros del mejor modelo
print('')
print('Mejores hiperparámetros:')
# Imprimimos los hiperparámetros del modelo del diccionario best_params de la forma: hiperparámetro: valor
for key, value in best_params.items():
    print(f'{key}: {value}')

#Guardamos el regresor XGBoost con los mejores hiperparámetros y pruning
with open('regressor_xgb_pruning.pkl', 'wb') as f:
    pickle.dump(best_model, f)

MAE con XGBoost Optimizado y Prunning: 1940.9989838382833
Número de trials: 140

Mejores hiperparámetros:
learning_rate: 0.09391585041713191
n_estimators: 956
max_depth: 10
max_leaves: 84
min_child_weight: 1
reg_alpha: 0.8105922236983762
reg_lambda: 0.4876393145933732
min_frequency: 0.05401631954675916


¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?

**Respuesta:**
```
Al incorporar el pruning durante la optimización, el número de trials aumenta de 135 a 140, lo que podría deberse a que el
entrenamiento se interrumpe para aquellos conjuntos de hiperparámetros que no están mostrando mejoras significativas, lo
que podría llevar a un (levemente) menor tiempo de ejecución al evitar entrenar para los conjuntos de hiperparámetros menos óptimos.
Por otro lado, los resultados mejoran significativamente, obteniéndose un MAE de 1940, lo que es mejor que los 2043 sin
pruning, por lo que se considera que este último modelo conlleva un aporte importante a la calidad de las predicciones y es el mejor
de los obtenidos en este laboratorio.
```

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [19]:
# Graficamos el historial de optimización
plot_optimization_history(study_opt_pr)

¿Desde qué trial se empiezan a observar mejoras notables en sus resultados?

**Respuesta:**
```
A partir del gráfico se puede observar que los resultados mejoran notablemente en varios puntos, por ejemplo pasando del trial 0 al
trial 1, luego en el trial 15 y finalmente en el trial 25. A partir de este punto siguen habiendo mejoras más leves hasta llegar al
trial 115 en que se alcanza el mejor valor de MAE entre todos los trials.
```

In [20]:
#Graficamos las coordenadas paralelas
plot_parallel_coordinate(study_opt_pr)

¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas?

**Respuesta:**
```
En el gráfico previo se puede ver que en varias de las variables las líneas se concentran en un rango reducido de valores, como lo
pueden ser learning_rate, reg_alpha y reg_lambda tendiendo a valores más altos, mientras que otras variables como
min_child_weight y min_frequency tienden a valores más bajo.
Esto puede evidenciar cómo el método de pruning poda las ramas que dan resultados poco efectivos, de modo que para los valores poco
óptimos de los hiperparámetros se encuentran pocas líneas, enfocándose mayoritariamente en aquellos valores que optimizan los
resultados.
```

In [21]:
#Graficamos la importancia de hiperparámetros
plot_param_importances(study_opt_pr)

¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

**Respuesta:**
```
De acuerdo a lo observado en el gráfico anterior se puede afirmar que las variables más importante son min_frequency que es la más
contribuyente al rendimiento del modelo liderando por un amplio margen. A esta variable le siguen learning_rate, y más atrás
reg_alpha, n_estimators y max depth.

Los gráficos de importancia de hiperparámetros reflejan cómo cada parámetro contribuye al rendimiento del modelo, evaluando su
impacto relativo. En el caso de la optimización con Optuna, el atributo de mayor importancia es min_frequency, seguido por
max_leaves y reg_lambda. Por otro lado, cuando se emplea el Pruning junto con Optuna, min_frequency también se destaca como el más
influyente, seguido por learning_rate y reg_alpha. En ambos casos, min_frequency se posiciona como el parámetro más significativo
en la optimización, aunque las posiciones relativas de otros hiperparámetros varían entre los métodos de optimización, lo que
sugiere que el Pruning puede dar mayor importancia a otros atributos como learning_rate y reg_alpha en la mejora del modelo.
```

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento?

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Por qué puede ocurrir esto?

|  | Baseline | XGBoost | XGBoost+Constraint | XGBoost+Optuna | XGBoost+pruning |
|-----------|-----------|-----------|-----------|-----------|-----------|
| MAE        | 13413.18        | 2427.11       | 2447.61        | 2043.05        | 1940.99        |


```
A partir de lo anterior y lo mencionado a lo largo del laboratorio se puede indicar que el mejor modelo fue el que utiliza el
pruning, obteniendo el mejor valor de MAE por un amplio margen respecto a los 3 modelos que le siguen, mejorando cerca de un 5%
respecto al modelo más cercano a él.
```

In [28]:
#Predecimos sobre los datos de test
test_predictions_xgb_best_pruning = best_model.predict(X_test)

#Obtenemos el valor del MAE
mae_xgb_best_pruning = mean_absolute_error(y_test, test_predictions_xgb_best_pruning)

#Imprimimos resultados relevantes
print(f'MAE con XGBoost y los mejores hiperparámetros (prunning): {mae_xgb_best_pruning}')
#Guardamos el regresor XGBoost con los mejores hiperparámetros y pruning
with open('regressor_xgb_best_pruning.pkl', 'wb') as f:
    pickle.dump(best_model, f)

MAE con XGBoost y los mejores hiperparámetros (prunning): 2095.3997821564803


```
El modelo entrega un MAE de 2095.39 en el conjunto de test, lo que es similar a lo obtenido sobre el conjunto de validation por el
modelo sin pruning.
La diferencia apreciada puede deberse a ligeras diferencias entre los conjuntos de validation y test, los que pese a ser similares,
no se adaptan perfectamente a lo aprendido por el modelo.
A pesar de presentar peores resultados sobre este conjunto de
datos, los resultados son considerados buenos, pues la diferencia no es excesivamente grande y aun así tiene mejores resultados que
los primeros tres modelos.
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>